# NLP 튜터리얼(Nonlinear Program (NLP) Tutorial)
notebook 실행 방법은 [index](./index.ipynb)를 참조하자.

## Important Note
Drake에서 일반적인 최적화 문제를 생성하고 풀기 위해 [MathematicalProgram Tutorial](./mathematical_program.ipynb)을 참조하자.

## Nonlinear Program
비선형 프로그램 (NLP) 문제는 특수한 유형의 최적화 문제이다. NLP에서 비용 또는 제약 조건은 결정 변수의 비선형 함수이다. 일반적인 NLP의 수학적 공식은 다음과 같다.

$\begin{aligned} \min_x&\; f(x)\\ \text{subject to }& g_i(x)\leq 0 \end{aligned}$

여기서 $f(x)$는 비용 함수이고, $g_i(x)$는 i번째 제약 조건이다.

일반적으로 NLP는 경사 기반 최적화(경사 하강, SQP, 내부 점 방법(interior point methods) 등)를 통해 해결한다. 이러한 방법들은 비용/제약 조건의 기울기에 의존합니다. 즉, $\partial f/\partial x, \partial g_i/\partial x$ 값들이 필요하다. pydrake는 자동 미분을 통해 많은 함수의 기울기를 계산할 수 있으므로 사용자가 직접 기울기를 제공할 필요가 거의 없다.

## Setting the objective
사용자는 `AddCost` 함수를 호출하여 프로그램에 비선형 비용을 해당 프로그램에 추가할 수 있다. 사용자는 `AddCost`를 반복적으로 호출할 수 있으며, 프로그램은 각 개별 비용의 *합*을 총 비용으로 평가(evaluation)한다.

### python 함수를 통해 비용 추가하기(Adding a cost through a python function)
파이썬 함수를 통해 비용을 정의한 다음, `AddCost` 함수를 사용하여 이 파이썬 함수를 목표 함수(objective)에 추가할 수 있다. 비용을 추가할 때는 해당 비용과 관련된 변수를 제공해야 한다. 문법은 `AddCost(cost_evaluator, vars=associated_variables)`이며, 이는 비용이 `associated_variables`에 대해 평가됨을 의미한다. 아래 코드 예제에서는 먼저 3개의 의사결정 변수로 최적화 프로그램을 구성하는 방법을 보여주고, 파이썬 함수를 통해 비용을 추가하는 방법을 보여준다.

In [ ]:
from pydrake.solvers import MathematicalProgram, Solve
import numpy as np

# Create an empty MathematicalProgram named prog (with no decision variables,
# constraints or costs)
prog = MathematicalProgram()
# Add three decision variables x[0], x[1], x[2]
x = prog.NewContinuousVariables(3, "x")

In [ ]:
def cost_fun(z):
    cos_z = np.cos(z[0] + z[1])
    sin_z = np.sin(z[0] + z[1])
    return cos_z**2 + cos_z + sin_z
# Add the cost evaluated with x[0] and x[1].
cost1 = prog.AddCost(cost_fun, vars=[x[0], x[1]])
print(cost1)

`AddCost` 함수의 `vars` 인수를 변경하면 다른 변수 집합에 비용을 추가할 수 있다. 아래 코드 예제에서는 동일한 파이썬 함수 `cost_fun`을 사용하지만 변수 `x[0], x[2]`에 이 비용을 부과한다.

In [ ]:
cost2 = prog.AddCost(cost_fun, vars=[x[0], x[2]])
print(cost2)

### 람다 함수로 비용 추가하기(Adding cost through a lambda function)
비용을 추가하는 더 간결한 방법은 람다 함수를 사용하는 것이다. 예를 들어, 아래 코드는 비용 $x[1]^2 + x[0]$을 최적화 프로그램에 추가한다.

In [ ]:
# Add a cost x[1]**2 + x[0] using a lambda function.
cost3 = prog.AddCost(lambda z: z[0]**2 + z[1], vars = [x[1], x[0]])
print(cost3)

연관된 변수를 변경하면 다른 비용을 나타낸다. 예를 들어, 동일한 람다 함수를 사용하지만 `vars`의 인자를 변경하여 비용 $x[1]^2 + x[2]$를 프로그램에 추가할 수 있다.

In [ ]:
cost4 = prog.AddCost(lambda z: z[0]**2 + z[1], vars = x[1:])
print(cost4)

## 2차 비용 추가하기(Adding quadratic cost)
NLP에서 $0.5x^TQx+ b'x+c$ 형태의 2차 비용(quadratic cost)을 추가하는 것은 매우 일반적이다. pydrake는 2차 비용을 추가하기 위해서 다음과 같은 다양한 함수를 제공한다.
- `AddQuadraticCost`
- `AddQuadraticErrorCost`
- `Add2NormSquaredCost`

### AddQuadraticCost
비용으로 간단한 2차 표현식을 추가할 수 있다.

In [ ]:
cost4 = prog.AddQuadraticCost(x[0]**2 + 3 * x[1]**2 + 2*x[0]*x[1] + 2*x[1] * x[0] + 1)
print(cost4)

만약 사용자가 `Q`와 `b` 행렬의 형태를 알고 있다면 위와 같은 상징적 2차식 표현보다는 직접 행렬을 넘겨주는 `AddQuadraticCost` 함수를 사용하는 것이 더 빠르다.

In [ ]:
# Add a cost x[0]**2 + 2*x[1]**2 + x[0]*x[1] + 3*x[1] + 1.
cost5 = prog.AddQuadraticCost(
    Q=np.array([[2., 1], [1., 4.]]),
    b=np.array([0., 3.]),
    c=1.,
    vars=x[:2])
print(cost5)

### AddQuadraticErrorCost
 이 함수는 $(x - x_{des})^TQ(x-x_{des})$ 형태의 비용을 추가한다.

In [ ]:
cost6 = prog.AddQuadraticErrorCost(
    Q=np.array([[1, 0.5], [0.5, 1]]),
    x_desired=np.array([1., 2.]),
    vars=x[1:])
print(cost6)

### Add2NormSquaredCost
이 함수는 $(Ax-b)^T(Ax-b)$ 형태의 2차 비용을 추가한다.

In [ ]:
# Add the L2 norm cost on (A*x[:2] - b).dot(A*x[:2]-b)
cost7 = prog.Add2NormSquaredCost(
    A=np.array([[1., 2.], [2., 3], [3., 4]]),
    b=np.array([2, 3, 1.]),
    vars=x[:2])
print(cost7)

## Adding constraints

Drake는 다음과 같은 형태의 제약 조건을 추가하는 것을 지원한다.
$$
\begin{aligned}
lower \leq g(x) \leq upper
\end{aligned}
$$
$g(x)$는 numpy vector를 반환한다.

사용자는 제약 조건을 추가하기 위해 `AddConstraint(g, lower, upper, vars=x)`를 호출할 수 있다. 여기서 `g`는 파이썬 함수(또는 람다 함수)여야 한다.

In [ ]:
## Define a python function to add the constraint x[0]**2 + 2x[1]<=1, -0.5<=sin(x[1])<=0.5
def constraint_evaluator1(z):
    return np.array([z[0]**2+2*z[1], np.sin(z[1])])

constraint1 = prog.AddConstraint(
    constraint_evaluator1,
    lb=np.array([-np.inf, -0.5]),
    ub=np.array([1., 0.5]),
    vars=x[:2])
print(constraint1)

# Add another constraint using lambda function.
constraint2 = prog.AddConstraint(
    lambda z: np.array([z[0]*z[1]]),
    lb=[0.],
    ub=[1.],
    vars=[x[2]])
print(constraint2)

## 비선형 프로그램 풀기(Solving the nonlinear program)

모든 제약 조건과 비용이 프로그램에 추가되면 `Solve` 함수를 호출하여 프로그램을 해결하고 `GetSolution`을 호출하여 결과를 얻을 수 있다. NLP를 해결하려면 모든 의사결정 변수에 대한 초기 추측이 필요하다. 사용자가 초기 추측을 지정하지 않으면 기본적으로 제로 벡터를 사용한다.

In [ ]:
## Solve a simple nonlinear 
# min               -x0
# subject to x1 - exp(x0) >= 0
#            x2 - exp(x1) >= 0
#            0 <= x0 <= 100
#            0 <= x1 <= 100
#            0 <= x2 <= 10
prog = MathematicalProgram()
x = prog.NewContinuousVariables(3)
# The cost is a linear function, so we call AddLinearCost
prog.AddLinearCost(-x[0])
# Now add the constraint x1-exp(x0)>=0 and x2-exp(x1)>=0
prog.AddConstraint(
    lambda z: np.array([z[1]-np.exp(z[0]), z[2]-np.exp(z[1])]),
    lb=[0, 0],
    ub=[np.inf, np.inf],
    vars=x)
# Add the bounding box constraint 0<=x0<=100, 0<=x1<=100, 0<=x2<=10
prog.AddBoundingBoxConstraint(0, 100, x[:2])
prog.AddBoundingBoxConstraint(0, 10, x[2])

# Now solve the program with initial guess x=[1, 2, 3]
result = Solve(prog, np.array([1.,2.,3.]))
print(f"Is optimization successful? {result.is_success()}")
print(f"Solution to x: {result.GetSolution(x)}")
print(f"optimal cost: {result.get_optimal_cost()}")

### 초기 추측 설정하기(Setting the initial guess)
일부 NLP 문제는 매우 많은 의사결정 변수를 가질 수 있습니다. 이러한 변수들의 초기 추측값을 설정하기 위해, pydrake는 `SetInitialGuess` 함수를 제공합니다. 이 함수는 일부 의사결정 변수에만 초기 추측값을 설정할 수 있습니다. 예를 들어, 아래 문제에서 우리는 단위 원 위의 점 $p_1$과 곡선 $y=x^2$ 위의 점 $p_2$ 중 가장 가까운 두 점을 찾고 싶습니다. 이 경우, `SetInitialGuess` 함수를 사용하여 $p_1$와 $p_2$ 에 각각 별도로 초기 추측값을 설정할 수 있다.

In [ ]:
import matplotlib.pyplot as plt
prog = MathematicalProgram()
p1 = prog.NewContinuousVariables(2, "p1")
p2 = prog.NewContinuousVariables(2, "p2")

# Add the constraint that p1 is on the unit circle centered at (0, 2)
prog.AddConstraint(
    lambda z: [z[0]**2 + (z[1]-2)**2],
    lb=np.array([1.]),
    ub=np.array([1.]),
    vars=p1)

# Add the constraint that p2 is on the curve y=x*x
prog.AddConstraint(
    lambda z: [z[1] - z[0]**2],
    lb=[0.],
    ub=[0.],
    vars=p2)

# Add the cost on the distance between p1 and p2
prog.AddQuadraticCost((p1-p2).dot(p1-p2))

# Set the value of p1 in initial guess to be [0, 1]
prog.SetInitialGuess(p1, [0., 1.])
# Set the value of p2 in initial guess to be [1, 1]
prog.SetInitialGuess(p2, [1., 1.])

# Now solve the program
result = Solve(prog)
print(f"Is optimization successful? {result.is_success()}")
p1_sol = result.GetSolution(p1)
p2_sol = result.GetSolution(p2)
print(f"solution to p1 {p1_sol}")
print(f"solution to p2 {p2_sol}")
print(f"optimal cost {result.get_optimal_cost()}")

# Plot the solution.
plt.figure()
plt.plot(np.cos(np.linspace(0, 2*np.pi, 100)), 2+np.sin(np.linspace(0, 2*np.pi, 100)))
plt.plot(np.linspace(-2, 2, 100), np.power(np.linspace(-2, 2, 100), 2))
plt.plot(p1_sol[0], p1_sol[1], '*')
plt.plot(p2_sol[0], p2_sol[1], '*')
plt.axis('equal')
plt.show()